In [17]:
# Importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score
from sklearn.utils import parallel_backend
from pyearth import Earth
from patsy import dmatrix
from sklearn.model_selection import GridSearchCV
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# Loading data
X_test = pd.read_csv('data/X_test.csv')
X_train = pd.read_csv('data/X_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_train = pd.read_csv('data/y_train.csv')

In [18]:
# Simultaneously optimizing hyperparameters
    # Max_depth and max_degree
param_grid = {    
    'max_terms': np.arange(2,27, 5),
    'max_degree': np.arange(100, 1500, 250)
}

grid_search = GridSearchCV(Earth(), param_grid, scoring="neg_mean_squared_error", n_jobs = -1, verbose = True)
grid_search.fit(X_train, y_train)

print('Best MSE Through Grid Search : %.3f'%grid_search.best_score_)
print(grid_search.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.

Best MSE Through Grid Search : -0.515
{'max_degree': 100, 'max_terms': 22}


In [19]:
mars_model = Earth(max_terms = 100, max_degree = 22)
mars_model.fit(X_train, y_train)

In [ ]:
print(mars_model.summary())

In [ ]:
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, mars_model.predict(X_train))))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, mars_model.predict(X_test))))